In [87]:
from utils import *
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
data = load_data('data/mnist_large.csv')

X_data = data[0]

Y_data = np.where(data[1]>5, 1, 0).astype(np.float64)

n = X_data.shape[1]
m = Y_data.size

X_r, Y_r, X_e, Y_e = train_test_split(X_data, Y_data, .8)

n_units = 100

run_no=0

tf.reset_default_graph()
run_no += 1
with tf.device('/cpu:0'):
    X = tf.placeholder(X_data.dtype, (None, n), 'features')
    Y = tf.placeholder(Y_data.dtype, (None, 1), 'label')
    L1 = tf.layers.dense(inputs=X, units=n_units, activation=tf.nn.sigmoid, name='H1')
    L2 = tf.layers.dense(inputs=L1, units=10, activation=tf.nn.sigmoid, name='H2')
#     regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
#     dropout = tf.layers.dropout(L1, 1)
#     weights = tf.get_variable('o_weights', (n_units, 1), X.dtype, tf.random_uniform_initializer())
#     logits = tf.m atmul(L1, weights)
    BC = tf.layers.dense(inputs=L2, units=1, name='OUT')#, kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1))  
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=BC))
    tf.summary.scalar('loss', loss)
    optimiser = tf.train.GradientDescentOptimizer(learning_rate=.1).minimize(loss)
    writer = tf.summary.FileWriter('data/.graph/run{}/train'.format(run_no), tf.get_default_graph())
    test_writer = tf.summary.FileWriter('data/.graph/run{}/test'.format(run_no))
    merged_smr = tf.summary.merge_all()
    inference = tf.sigmoid(BC)

epochs = 500
losses = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
#         print(loss)
        smr = sess.run(merged_smr, feed_dict={X: X_r, Y: Y_r})
        writer.add_summary(smr, i)
        smr = sess.run(merged_smr, feed_dict={X: X_e, Y: Y_e})
        test_writer.add_summary(smr, i)
        sess.run(optimiser, feed_dict={X: X_r, Y: Y_r})
    prd = sess.run(inference, feed_dict={X: X_e, Y: Y_e})
writer.close()

prd = np.where(prd>.5, 1, 0)

print('ANN f1-score =', f1_score(Y_e, prd))

clf = KNeighborsClassifier(n_neighbors=3, weights='distance')
clf.fit(X_r, Y_r.astype(int))

k_prd = clf.predict(X_e)

print('KNN f1-score =', f1_score(Y_e.astype(int), k_prd))